In [2]:
import warnings; warnings.filterwarnings("ignore");
import time
start_time = time.time()

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
import re
#import enchant
import random
random.seed(2016)

In [3]:
df_train = pd.read_csv('train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('product_descriptions.csv')
df_attr = pd.read_csv('attributes.csv')
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
num_train = df_train.shape[0]
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
print("--- Files Loaded: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Files Loaded: 0.28 minutes ---


In [4]:
stop_w = ['for', 'xbi', 'and', 'in', 'th','on','sku','with','what','from','that','less','er','ing'] #'electr','paint','pipe','light','kitchen','wood','outdoor','door','bathroom'
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}


In [5]:
def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

In [6]:
def seg_words(str1, str2):
    str2 = str2.lower()
    str2 = re.sub("[^a-z0-9./]"," ", str2)
    str2 = [z for z in set(str2.split()) if len(z)>2]
    words = str1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s1 = []
            s1 += segmentit(word,str2,True)
            if len(s)>1:
                s += [z for z in s1 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))
def segmentit(s, txt_arr, t):
    st = s
    r = []
    for j in range(len(s)):
        for word in txt_arr:
            if word == s[:-j]:
                r.append(s[:-j])
                #print(s[:-j],s[len(s)-j:])
                s=s[len(s)-j:]
                r += segmentit(s, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [7]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['id','relevance','search_term','product_title','product_description','product_info','attr','brand']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [8]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
print("--- Stemming: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
print("--- Prod Info: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
print("--- Len of: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))
#print("--- Search Term Segment: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
print("--- Query In: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
print("--- Query Last Word In: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_brand = pd.unique(df_all.brand.ravel())


--- Stemming: 8.3 minutes ---
--- Prod Info: 8.31 minutes ---
--- Len of: 8.36 minutes ---
--- Query In: 8.53 minutes ---
--- Query Last Word In: 8.56 minutes ---


In [9]:
i = 1000
d={}
for s in df_brand:
    d[s]=i
    i+=3
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))
df_all.to_csv('df_all.csv')
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train =df_train[:]
X_test = df_test[:]
print("--- Features Set: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Features Set: 8.86 minutes ---


In [12]:
rfr = RandomForestRegressor(n_estimators = 700, n_jobs = 1, random_state = 2000, verbose = 20)
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=100, random_state = 2000)
clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        'txt3': 0.0,
                        'txt4': 0.5
                        },
                #n_jobs = -1
                )), 
        ('rfr', rfr)])
param_grid = {'rfr__max_features': [100], 'rfr__max_depth': [20]}
model = grid_search.GridSearchCV(estimator = clf, param_grid = param_grid,
                                 n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)
print("done")

done


In [13]:
#AdaBoostRegressor
abr = AdaBoostRegressor()
clf_abr = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        'txt3': 0.0,
                        'txt4': 0.5
                        },
                #n_jobs = -1
                )), 
        ('abr', abr)])
#param_grid = {'abr__max_features': [100], 'abr__max_depth': [20]}
#model_abr = grid_search.GridSearchCV(estimator = clf_abr, param_grid = param_grid,
#                                 n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)

clf_abr.fit(X_train, y_train)
y_abr_pred = clf_abr.predict(X_test)
pd.DataFrame({"id": id_test, "relevance": y_abr_pred}).to_csv('submission_abr.csv',index=False)
print("--- Training & Testing: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Training & Testing: 25.77 minutes ---


In [ ]:
print (param_grid)

In [14]:
#model.get_params().keys()
model.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rfr__max_depth=20, rfr__max_features=100 ........................
building tree 1 of 700
building tree 2 of 700

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    4.1s



building tree 3 of 700
building tree 4 of 700

[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    6.4s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    8.5s



building tree 5 of 700
building tree 6 of 700

...


building tree 698 of 700
building tree 699 of 700

[Parallel(n_jobs=1)]: Done 698 tasks       | elapsed: 61.5min
[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed: 61.6min



building tree 700 of 700


[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed: 61.7min
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed: 61.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('cst', cust_regression_vals()), ('txt1', Pipeline(steps=[('s1', cust_txt_col(key='search_term')), ('tfidf1', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', ...mators=700, n_jobs=1, oob_score=False, random_state=2000,
           verbose=20, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'rfr__max_depth': [20], 'rfr__max_features': [100]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(fmean_squared_error, greater_is_better=False),
       verbose=20)

In [ ]:
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)
print(model.best_score_ + 0.47003199274)
print(model)


In [15]:
y_pred = model.predict(X_test)
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_rfr_700_run.csv',index=False)
print("--- Training & Testing: %s minutes ---" % round(((time.time() - start_time)/60),2))

#print average of two classifier
pd.DataFrame({"id":id_test, "relevance": (y_pred+y_abr_pred)/2}).to_csv('submission_rfr_abr.csv', index=False)

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   5 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   6 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done   8 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done   9 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  10 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  11 tasks       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  13 tasks       | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  14 tasks       | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  15 tasks       | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  16 tasks       | elapsed:    0.7s
[Paralle

--- Training & Testing: 149.86 minutes ---


[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   37.0s finished


In [ ]:
X_train.to_csv('c2.csv')

In [18]:
#model 1
rfr = RandomForestRegressor(n_estimators = 700, n_jobs = 1, random_state = 2000, verbose = 20)
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=100, random_state = 2000)
clf1 = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 1.0,
                        'txt2': 1.0
                        },
                #n_jobs = -1
                )), 
        ('rfr', rfr)])
param_grid = {'rfr__max_features': [100], 'rfr__max_depth': [20]}
model = grid_search.GridSearchCV(estimator = clf1, param_grid = param_grid,
                                 n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)
print("done")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

done
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rfr__max_depth=20, rfr__max_features=100 ........................
building tree 1 of 700
building tree 2 of 700

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    3.1s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    6.5s



building tree 3 of 700
building tree 4 of 700

[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    9.8s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:   13.1s



building tree 5 of 700
building tree 6 of 700

...


building tree 698 of 700
building tree 699 of 700

[Parallel(n_jobs=1)]: Done 698 tasks       | elapsed: 117.4min
[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed: 117.5min



building tree 700 of 700

[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed: 117.6min
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed: 117.6min finished
[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   5 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   6 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   8 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   9 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  10 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  11 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  13 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  14 tasks       | elapsed:    0.5

[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed:   27.5s
[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed:   27.5s
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   27.5s finished


In [19]:
#model 2
rfr = RandomForestRegressor(n_estimators = 700, n_jobs = 1, random_state = 2000, verbose = 20)
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=100, random_state = 2000)
clf2 = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 1.0,
                        'txt3':1.0
                        },
                #n_jobs = -1
                )), 
        ('rfr', rfr)])
param_grid = {'rfr__max_features': [100], 'rfr__max_depth': [20]}
model2 = grid_search.GridSearchCV(estimator = clf2, param_grid = param_grid,
                                 n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)
print("done")
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

done
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rfr__max_depth=20, rfr__max_features=100 ........................
building tree 1 of 700
building tree 2 of 700

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    3.1s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    6.2s



building tree 3 of 700
building tree 4 of 700

[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    9.4s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:   12.5s



building tree 5 of 700
building tree 6 of 700

...


building tree 698 of 700
building tree 699 of 700

[Parallel(n_jobs=1)]: Done 698 tasks       | elapsed: 81.1min
[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed: 81.3min



building tree 700 of 700

[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed: 81.4min
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed: 81.4min finished
[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   5 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   6 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   8 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   9 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  10 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  11 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  13 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  14 tasks       | elapsed:    0.5s


[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed:   28.5s
[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed:   28.6s
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   28.6s finished


In [20]:
#model 3
rfr = RandomForestRegressor(n_estimators = 700, n_jobs = 1, random_state = 2000, verbose = 20)
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=100, random_state = 2000)
clf3 = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 1.0,
                        'txt4': 1.0
                        },
                #n_jobs = -1
                )), 
        ('rfr', rfr)])
param_grid = {'rfr__max_features': [100], 'rfr__max_depth': [20]}
model3 = grid_search.GridSearchCV(estimator = clf3, param_grid = param_grid,
                                 n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)
print("done")
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)

done
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rfr__max_depth=20, rfr__max_features=100 ........................
building tree 1 of 700
building tree 2 of 700

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    4.6s



building tree 3 of 700
building tree 4 of 700

[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    7.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    9.3s



building tree 5 of 700
building tree 6 of 700

...


building tree 698 of 700
building tree 699 of 700

[Parallel(n_jobs=1)]: Done 698 tasks       | elapsed: 60.3min
[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed: 60.4min



building tree 700 of 700

[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed: 60.5min
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed: 60.5min finished
[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   5 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   6 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   8 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done   9 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  10 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  11 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  13 tasks       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  14 tasks       | elapsed:    0.5s


[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed:   29.5s finished


In [21]:
#model 4
rfr = RandomForestRegressor(n_estimators = 700, n_jobs = 1, random_state = 2000, verbose = 20)
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=100, random_state = 2000)
clf4 = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()), 
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt2': 1.0,
                        'txt3': 1.0,
                        'txt4': 1.0
                        },
                #n_jobs = -1
                )), 
        ('rfr', rfr)])
param_grid = {'rfr__max_features': [100], 'rfr__max_depth': [20]}
model4 = grid_search.GridSearchCV(estimator = clf4, param_grid = param_grid,
                                 n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)
print("done")
model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)

done
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rfr__max_depth=20, rfr__max_features=100 ........................
building tree 1 of 700
building tree 2 of 700

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    3.3s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    6.3s



building tree 3 of 700
building tree 4 of 700

[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    9.4s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:   12.3s



building tree 5 of 700
building tree 6 of 700

...

[Parallel(n_jobs=1)]: Done 696 tasks       | elapsed: 76.4min
[Parallel(n_jobs=1)]: Done 697 tasks       | elapsed: 76.5min



building tree 698 of 700
building tree 699 of 700

[Parallel(n_jobs=1)]: Done 698 tasks       | elapsed: 76.6min
[Parallel(n_jobs=1)]: Done 699 tasks       | elapsed: 76.7min



building tree 700 of 700

[Parallel(n_jobs=1)]: Done 700 tasks       | elapsed: 76.8min
[Parallel(n_jobs=1)]: Done 700 out of 700 | elapsed: 76.8min finished
[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   5 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   6 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   8 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done   9 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  10 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  11 tasks       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  13 tasks       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  14 tasks       | elapsed:    0.6s


In [22]:
#print ensemble result
pd.DataFrame({"id": id_test, "relevance": (y_pred+y_pred2+y_pred3+y_pred4)/4}).to_csv('submission_rfr_ensemble_4splitF.csv',index=False)
print("--- Training & Testing: %s minutes ---" % round(((time.time() - start_time)/60),2))

--- Training & Testing: 793.18 minutes ---
